In [21]:
# sequential workflow

from langgraph.graph import StateGraph, START, END
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from typing import TypedDict
from dotenv import load_dotenv

In [22]:
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id = "mistralai/Mistral-7B-Instruct-v0.2",
    task = "text-generation"
)

model = ChatHuggingFace(llm=llm)

In [23]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str
    score: int

In [24]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm and generate outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [25]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [26]:
def evaluate(state: BlogState) -> BlogState:
    outline = state['outline']
    content = state['content']

    prompt = f'Based on the outline {outline}, rate the score of blog {content} between 0 to 10.'
    score = model.invoke(prompt).content
    state['score'] = score
    return state

In [27]:

graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluate_blog', evaluate)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluate_blog')
graph.add_edge('evaluate_blog', END)

workflow = graph.compile()

In [28]:
intial_state = {'title': 'Rise of Indian Cricket from 90s to modern cricket'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'Rise of Indian Cricket from 90s to modern cricket', 'outline': " Title: Rise of Indian Cricket: From the 90s to Modern-Day Glory\n\nI. Introduction\n  A. Brief overview of Indian cricket before the 90s\n  B. Importance of cricket in Indian culture\n  C. Thesis statement: The 90s marked a turning point in Indian cricket, leading to modern-day success\n\nII. Early 90s: The struggle for recognition\n  A. Challenges faced by Indian cricket in the early 90s\n     1. Lack of consistent victory\n     2. Corruption allegations\n     3. Poor team morale\n  B. The Azharuddin factor: Leadership and reform\n\nIII. Mid-90s: The rise of Kapil Dev and Sourav Ganguly\n  A. Kapil Dev's influence in transforming the team\n     1. As a player\n     2. As a captain\n     3. Legacy\n  B. Sourav Ganguly: The catalyst for change\n     1. First step towards modern-day success\n     2. Innovative leadership style\n     3. Impact on team morale and Indian cricket in general\n\nIV. Late 90s and early 

In [29]:
print(final_state['outline'])

 Title: Rise of Indian Cricket: From the 90s to Modern-Day Glory

I. Introduction
  A. Brief overview of Indian cricket before the 90s
  B. Importance of cricket in Indian culture
  C. Thesis statement: The 90s marked a turning point in Indian cricket, leading to modern-day success

II. Early 90s: The struggle for recognition
  A. Challenges faced by Indian cricket in the early 90s
     1. Lack of consistent victory
     2. Corruption allegations
     3. Poor team morale
  B. The Azharuddin factor: Leadership and reform

III. Mid-90s: The rise of Kapil Dev and Sourav Ganguly
  A. Kapil Dev's influence in transforming the team
     1. As a player
     2. As a captain
     3. Legacy
  B. Sourav Ganguly: The catalyst for change
     1. First step towards modern-day success
     2. Innovative leadership style
     3. Impact on team morale and Indian cricket in general

IV. Late 90s and early 2000s: Consolidating victories
  A. Major milestones and series victories
     1. 1996 World Cup an

In [30]:
print(final_state['content'])

 Title: Rise of Indian Cricket: From the 90s to Modern-Day Glory

I. Introduction
Indian cricket, a passionate pursuit deeply rooted in the cultural fabric of the country, was on the brink of a transformative era in the 1990s. Through this blog, we will delve into the pivotal decade that shaped Indian cricket into the modern-day powerhouse it is known as today. Our journey begins with a brief overview of Indian cricket before the 90s, the significance of cricket in Indian culture, and a thesis statement that the 90s marked a turning point for Indian cricket.

II. Early 90s: The struggle for recognition
A. The 90s brought new challenges to Indian cricket. Being a time when victory was elusive, the team grappled with countless issues; from lack of consistent wins to corruption allegations and poor team morale.

B. Mohammad Azharuddin's tenure as captain came at a crucial time. His leadership brought about reforms, helped restore the team's pride, and instilled a new sense of determinatio

In [31]:
print(final_state.keys())

dict_keys(['title', 'outline', 'content', 'score'])


In [32]:
print(final_state['score'])

 Based on the content and organization of the blog, I would rate it an 8 out of 10. The blog provides a comprehensive overview of the significant milestones and influences in Indian cricket from the 90s to the modern day. The writing is engaging and easy to follow, making it an informative read for cricket enthusiasts. However, some areas could be improved, such as expanding upon the challenges faced during the early 90s and further elaborating on the BCCI's role. Additionally, a more robust analysis of the impact of T20 cricket and the IPL on modern-day Indian cricket would add value to the blog. Overall, it is an impressive piece of work that successfully encapsulates the journey of Indian cricket over the past few decades.
